In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # Import LLM class
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""  # Replace with your API key

# Initialize the LLM
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Choose your model and parameters

!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'pg_essay.txt'


# Load documents (Replace with your own data loading logic)
def load_documents():
    loader = TextLoader("pg_essay.txt")  # Path to your text file
    documents = loader.load()
    return documents

# Perform semantic chunking
def semantic_chunking(documents, chunk_size=512, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = []
    for doc in documents:
        chunks.extend(text_splitter.split_text(doc.page_content))
    return chunks

# Embed and store the chunks
def embed_and_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store

# Main RAG pipeline
def rag_pipeline():
    # Load and chunk documents
    documents = load_documents()
    chunks = semantic_chunking(documents)

    # Embed and store
    vector_store = embed_and_store(chunks)

    # Set up retriever
    retriever = vector_store.as_retriever()

    # Initialize LLM
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=256
    )

    # Set up a QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,  # Provide the language model
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=True
    )

    return qa_chain

if __name__ == "__main__":
    qa_chain = rag_pipeline()
    query = "What is semantic chunking?"
    
    # Use __call__ instead of run to get all outputs
    result = qa_chain({"query": query})  # Pass the query as a dictionary
    
    # Extract the results
    answer = result["result"]
    source_documents = result["source_documents"]
    
    # Print the outputs
    print("Answer:", answer)
    print("\nSource Documents:")
    for doc in source_documents:
        print(doc.page_content)